
# 🧩 Product Recommendation Systems — MF (ALS) + Quantum-Inspired QUBO (Colab-Ready)

This notebook demonstrates a hybrid **classical–quantum approach** to product recommendations on a **synthetic dataset**:

1. **Classical Matrix Factorization (ALS)**  
   - Implicit feedback (click/purchase) dataset  
   - Alternating Least Squares (ALS) with L2 regularization  
   - Evaluation: **HitRate@K** and **NDCG@K**

2. **Quantum-Inspired Boltzmann / Ising QUBO**  
   - Learn **item–item couplings** from co-occurrence (PMI-like).  
   - Combine **MF scores** (linear fields) and couplings with a **Top-N constraint**.  
   - Solve with `neal` (simulated annealing).

Both use the same dataset for fair comparison.


In [ ]:

# Install dependencies (dimod + neal) if missing
def _silent_imports():
    flags = {"dimod": False, "neal": False}
    try:
        import dimod
        flags["dimod"] = True
    except Exception:
        pass
    try:
        import neal
        flags["neal"] = True
    except Exception:
        pass
    return flags

flags = _silent_imports()
if not flags["dimod"] or not flags["neal"]:
    %pip -q install dimod neal

flags = _silent_imports()
print("dimod:", flags["dimod"], "| neal:", flags["neal"])


In [ ]:

# ==== Synthetic User–Item Interaction Dataset ====
import numpy as np, pandas as pd

rng = np.random.default_rng(2028)

U = 400     # users
I = 250     # items
K = 10      # latent factors
density = 0.05

# Latent factors (true)
U_true = rng.normal(0, 1, (U, K))
V_true = rng.normal(0, 1, (I, K))
bias_item = rng.normal(0, 0.3, size=I)

# Generate implicit feedback (Bernoulli with sigmoid)
def sigmoid(x): return 1 / (1 + np.exp(-x))
score = U_true @ V_true.T + bias_item[None, :]
prob = sigmoid(score)
mask = rng.random((U, I)) < density
Y = (rng.random((U, I)) < prob) & mask
Y = Y.astype(int)

# Leave-one-out test
train = Y.copy()
test_pos = -np.ones(U, dtype=int)
for u in range(U):
    pos = np.where(Y[u] == 1)[0]
    if len(pos) > 0:
        hold = rng.choice(pos)
        train[u, hold] = 0
        test_pos[u] = hold

print(f"Users={U}, Items={I}, Positives={Y.sum()}")



## Part 1 — Matrix Factorization (ALS)


In [ ]:

# ALS implementation (no external dependencies)
import numpy as np

def als_implicit(train, K=10, reg=0.1, iters=10, seed=0):
    rng = np.random.default_rng(seed)
    U, I = train.shape
    P = rng.normal(0, 0.1, (U, K))
    Q = rng.normal(0, 0.1, (I, K))
    I_K = reg * np.eye(K)
    for _ in range(iters):
        # Update P
        QTQ = Q.T @ Q + I_K
        for u in range(U):
            idx = np.where(train[u] == 1)[0]
            if len(idx) == 0: continue
            Q_u = Q[idx]
            A = QTQ + Q_u.T @ Q_u - I_K
            b = Q_u.T @ np.ones(len(idx))
            P[u] = np.linalg.solve(A, b)
        # Update Q
        PTP = P.T @ P + I_K
        for i in range(I):
            idx = np.where(train[:, i] == 1)[0]
            if len(idx) == 0: continue
            P_i = P[idx]
            A = PTP + P_i.T @ P_i - I_K
            b = P_i.T @ np.ones(len(idx))
            Q[i] = np.linalg.solve(A, b)
    return P, Q

P, Q = als_implicit(train, K=K, reg=0.2, iters=12)
S_mf = P @ Q.T
print("MF mean/std scores:", round(S_mf.mean(), 4), round(S_mf.std(), 4))


In [ ]:

# Evaluate HitRate@K and NDCG@K
def evaluate_topk(S, train, test_pos, Krec=10, n_negs=100, seed=0):
    rng = np.random.default_rng(seed)
    U, I = S.shape
    hits = ndcg = 0
    valid_users = 0
    for u in range(U):
        tp = test_pos[u]
        if tp < 0: continue
        negs = np.setdiff1d(np.arange(I), np.concatenate([[tp], np.where(train[u]==1)[0]]))
        if len(negs) < n_negs: continue
        neg_sample = rng.choice(negs, n_negs, replace=False)
        cand = np.concatenate([[tp], neg_sample])
        scores = S[u, cand]
        order = np.argsort(-scores)
        rank = np.where(order == 0)[0][0]
        if rank < Krec:
            hits += 1
            ndcg += 1 / np.log2(rank + 2)
        valid_users += 1
    return hits / valid_users, ndcg / valid_users

hit10, ndcg10 = evaluate_topk(S_mf, train, test_pos)
print("MF HitRate@10:", round(hit10, 4), "| NDCG@10:", round(ndcg10, 4))



## Part 2 — Quantum-Inspired QUBO Recommendation


In [ ]:

from collections import defaultdict
import dimod, neal

# Compute co-occurrence matrix for PMI-like coupling
item_pop = train.mean(axis=0)
pair_pop = (train.T @ train) / train.shape[0]
P_i = item_pop
P_ij = pair_pop / pair_pop.max()
np.fill_diagonal(P_ij, P_i)

PMI = np.log((P_ij + 1e-9) / (P_i[:, None] * P_i[None, :] + 1e-9))
PMI = np.maximum(0.0, PMI)
np.fill_diagonal(PMI, 0.0)
h_item = np.log((P_i + 1e-6) / (1 - P_i + 1e-6))

def qubo_recommend(u, S_mf, Nrec=10, M_cand=80, alpha=1.0, beta=0.2, gamma=0.6, A=6.0, seed=0):
    rng = np.random.default_rng(seed)
    seen = np.where(train[u] == 1)[0]
    unseen = np.setdiff1d(np.arange(I), seen)
    if len(unseen) == 0: return []
    scores = S_mf[u, unseen]
    top_idx = np.argsort(-scores)[:min(M_cand, len(unseen))]
    cand = unseen[top_idx]
    m = len(cand)

    Q = defaultdict(float)
    for a, i in enumerate(cand):
        Q[(a, a)] += - (alpha * S_mf[u, i] + beta * h_item[i])
    for a, i in enumerate(cand):
        for b in range(a + 1, m):
            j = cand[b]
            Q[(a, b)] += - gamma * PMI[i, j]
    for a in range(m):
        Q[(a, a)] += A * (1 - 2 * Nrec)
    for a in range(m):
        for b in range(a + 1, m):
            Q[(a, b)] += 2 * A

    bqm = dimod.BinaryQuadraticModel.from_qubo(dict(Q))
    ss = neal.SimulatedAnnealingSampler().sample(bqm, num_reads=800, seed=seed)
    best = ss.first.sample
    chosen = [cand[a] for a in range(m) if best.get(a, 0) == 1]

    if len(chosen) < Nrec:
        pad = [i for i in cand if i not in chosen]
        pad = pad[:Nrec - len(chosen)]
        chosen += pad
    elif len(chosen) > Nrec:
        chosen = chosen[:Nrec]
    return chosen

# Example
examples = {u: qubo_recommend(u, S_mf, seed=42 + u) for u in range(3)}
examples


In [ ]:

# Compare QUBO vs MF on test positives
def eval_user(u, rec_list, test_pos):
    tp = test_pos[u]
    if tp < 0: return 0, 0
    if tp in rec_list:
        rank = rec_list.index(tp)
        return 1, 1 / np.log2(rank + 2)
    return 0, 0

users_eval = 200
hits_mf = ndcg_mf = hits_qb = ndcg_qb = 0
rng = np.random.default_rng(1)
users = rng.choice([u for u in range(U) if test_pos[u] >= 0], size=users_eval, replace=False)

for u in users:
    rec_mf = np.argsort(-S_mf[u])
    rec_mf = [i for i in rec_mf if train[u, i] == 0][:10]
    rec_qb = qubo_recommend(u, S_mf, seed=u + 10)
    h, g = eval_user(u, rec_mf, test_pos); hits_mf += h; ndcg_mf += g
    h, g = eval_user(u, rec_qb, test_pos); hits_qb += h; ndcg_qb += g

print("MF  @10 — Hit:", round(hits_mf / users_eval, 4), "NDCG:", round(ndcg_mf / users_eval, 4))
print("QUBO@10 — Hit:", round(hits_qb / users_eval, 4), "NDCG:", round(ndcg_qb / users_eval, 4))



### Notes
- **Matrix Factorization** provides linear predictions from latent factors.  
- The **QUBO** step acts as a **Boltzmann machine**, coupling items via PMI co-occurrence.  
- You can tune `gamma` for coherence (positive = similar items, negative = diverse).  
- The QUBO structure can extend to **quantum annealers** or **tensor-network samplers** for large-scale factorization.
